# Linear equation system solving techniques comparison



In [2]:
import numpy as np
import math as mt


<h4>
    <i>
        index = 172110 </br>
        c = 1
        </br>
        d = 0
        </br>
        e = 1 
        </br>
        f = 2
        </br>
        9cd = 910
    </i>
</h4>

**Exercise A**


In [1]:
N = 910
f = 2
e = 1

a1 = 5 + e
a2 = -1
a3 = a2 

iter_limit = 1_000_000

<h4>Helper function</h4>


In [3]:
def striped_matrix_value(x:int, y:int) -> float:
    if x == y:
        return a1
    if x == y - 1:
        return a2
    if x == y - 2:
        return a3
    
    if x == y + 1:
        return a2
    if x == y + 2:
        return a3
    return 0



<h4>
Matrix initialization
</h4>


In [12]:
B = np.ndarray(shape=(N, 1), dtype=float)
for i in range(N):
    B[i] = mt.sin(i * (f + 1))
        
A = np.ndarray(shape=(N, N), dtype=float)
for i in range(N):
    for j in range(N):
        A[i][j] = striped_matrix_value(i,j)

# Solution matrix
x = np.ndarray(shape=(N, 1), dtype=float)
for i in range(N):
    x[i] = 0


**Exercise B**

<h3>Jacobi's method naive way</h3>

In [9]:
def jacobis_next_iteration(matrix: np.ndarray, b_vec: np.ndarray, results: np.ndarray) -> np.ndarray:
    new_result: np.ndarray = np.ndarray(shape=(results.shape[0],1))
    for i in range(matrix.shape[0]):
        # Copy of the i-th equation
        eq = matrix[i].copy()
        # The coefficient of the x which we're calculating new value for
        curr_x_coeff = eq[i]
        # starting with the opposite of the excitement vector element
        new_x = - b_vec[i]
        for j in range(eq.shape[0]):
            # for every variable except the one we're currently calculating
            if j != i:
                # eq[j] is the coefficient of x(j) and results[j] is the previous value of the x(j)
                new_x += eq[j] * results[j]
        new_result[i] = new_x / -curr_x_coeff
    return new_result


<h2>Residue vector second norm calculation</h2>


In [10]:
def calculate_residue_vector_sec_norm(matrix: np.ndarray, est_sol: np.ndarray, b_vec: np.ndarray) -> float:
    n_size = matrix.shape[0]
    residue_vec_norm = 0.0
    for i in range(n_size):
        residue_vec_norm += (matrix[i][i] * est_sol[i] - b_vec[i]) ** 2
    return mt.sqrt(residue_vec_norm)
    
            

**Testing the Jacobis method on small data**

In [13]:
A_sm = np.ndarray(shape=(2,2), dtype=float)
A_sm[0] = [2,1]
A_sm[1] = [5,7]

B_sm = np.array([11.0, 13.0])

x_sm = np.array([1.0, 1.0])

for i in range(25):
    x_sm = jacobis_next_iteration(A_sm, B_sm, x_sm)
    print("x1: ",x_sm[0] , " x2: ",x_sm[1])
    

x1:  [5.]  x2:  [1.14285714]
x1:  [4.92857143]  x2:  [-1.71428571]
x1:  [6.35714286]  x2:  [-1.66326531]
x1:  [6.33163265]  x2:  [-2.68367347]
x1:  [6.84183673]  x2:  [-2.6654519]
x1:  [6.83272595]  x2:  [-3.02988338]
x1:  [7.01494169]  x2:  [-3.02337568]
x1:  [7.01168784]  x2:  [-3.15352978]
x1:  [7.07676489]  x2:  [-3.1512056]
x1:  [7.0756028]  x2:  [-3.19768921]
x1:  [7.0988446]  x2:  [-3.19685914]
x1:  [7.09842957]  x2:  [-3.21346043]
x1:  [7.10673022]  x2:  [-3.21316398]
x1:  [7.10658199]  x2:  [-3.21909301]
x1:  [7.10954651]  x2:  [-3.21898714]
x1:  [7.10949357]  x2:  [-3.22110465]
x1:  [7.11055232]  x2:  [-3.22106683]
x1:  [7.11053342]  x2:  [-3.22182309]
x1:  [7.11091154]  x2:  [-3.22180958]
x1:  [7.11090479]  x2:  [-3.22207967]
x1:  [7.11103984]  x2:  [-3.22207485]
x1:  [7.11103743]  x2:  [-3.22217131]
x1:  [7.11108566]  x2:  [-3.22216959]
x1:  [7.11108479]  x2:  [-3.22220404]
x1:  [7.11110202]  x2:  [-3.22220342]



<h2>Testing the Jacobi's method on given data</h2>


In [ ]:
i = 0
while calculate_residue_vector_sec_norm(A, x, B) or i == iter_limit:
    i += 1
    x = jacobis_next_iteration(A, B, x)
print("Residue vector second norm value: ",calculate_residue_vector_sec_norm(A,x, B))
print("Iteration count: ", i)

    

